# Data Cleaning
Clean the raw data collected from the scrapper and prepare the data ready for exploratory data analysis

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df = pd.read_csv("autotrader_prices_raw.csv", encoding= 'unicode_escape')

In [2]:
df.head()

,BHP,ULEZ,body,engine,fuel,link,mileage,name,owners,price,transmission,year
0,123.0,ULEZ,Hatchback,1.4L,Petrol,https://www.autotrader.co.uk/car-details/20201...,34414,"Audi A1 1.4 TFSI SPORT 3d 123 BHP, SPORTS SEAT...",NaN,"??9,980",Manual,2017 (17 reg)
1,NaN,ULEZ,Hatchback,1.4L,Petrol,https://www.autotrader.co.uk/car-details/20201...,16773,Audi A1 TFSI Sport 3dr 1.4,NaN,"??11,500",Manual,2017 (67 reg)
2,114.0,ULEZ,Hatchback,1.6L,Diesel,https://www.autotrader.co.uk/car-details/20200...,76922,Audi A1 1.6 TDI SPORT 3d 114 BHP PARKING SENSO...,NaN,"??8,599",Manual,2017 (17 reg)
3,NaN,ULEZ,Hatchback,1.4L,Petrol,https://www.autotrader.co.uk/car-details/20201...,10950,Audi A1 1.4 TFSI Sport 5dr,NaN,"??12,174",Manual,2017 (67 reg)
4,NaN,ULEZ,Hatchback,1.6L,Diesel,https://www.autotrader.co.uk/car-details/20201...,58278,Audi A1 1.6 TDI Sport 3dr,NaN,"??9,172",Manual,2017 (67 reg)


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1233 entries, 0 to 1232
Data columns (total 12 columns):
BHP             773 non-null float64
ULEZ            1223 non-null object
body            1233 non-null object
engine          1227 non-null object
fuel            1232 non-null object
link            1233 non-null object
mileage         1233 non-null int64
name            1233 non-null object
owners          17 non-null float64
price           1233 non-null object
transmission    1232 non-null object
year            1233 non-null object
dtypes: float64(2), int64(1), object(9)
memory usage: 115.7+ KB


In [4]:
df.describe()

,BHP,mileage,owners
count,773.000000,1233.000000,17.000000
mean,154.737387,19593.000000,2.058824
std,58.945633,13701.411563,0.242536
min,68.000000,10.000000,2.000000
25%,114.000000,10665.000000,2.000000
50%,138.000000,16805.000000,2.000000
75%,181.000000,26179.000000,2.000000
max,469.000000,124395.000000,3.000000


In [5]:
df.isna().sum()

BHP              460
ULEZ              10
body               0
engine             6
fuel               1
link               0
mileage            0
name               0
owners          1216
price              0
transmission       1
year               0
dtype: int64

## Data cleaning

In [6]:
# create a copy of the dataframe
df_clean = df.copy()

### Filling ULEZ compliance
Assuming Non-ULEZ if information missing in the "ULEZ" column

In [7]:
df_clean["ULEZ"].fillna("Non-ULEZ", inplace=True)

In [8]:
df_clean["ULEZ"].value_counts()

ULEZ        1223
Non-ULEZ      10
Name: ULEZ, dtype: int64

### Engine size parsing
* Fill the missingg values
* Get rid of the Liter "L" and convert to float

In [9]:
# locate missing values
df_clean[df_clean.engine.isna()]

,BHP,ULEZ,body,engine,fuel,link,mileage,name,owners,price,transmission,year
295,201.0,ULEZ,SUV,NaN,Electric,https://www.autotrader.co.uk/car-details/20201...,23946,Hyundai Kona Premium SE 5dr,NaN,"??27,125",Automatic,2018 (68 reg)
328,NaN,Non-ULEZ,SUV,NaN,Electric,https://www.autotrader.co.uk/car-details/20200...,6295,Hyundai Kona Electric Premium SE (204PS) 64kWh...,NaN,"??31,890",Automatic,2020 (69 reg)
583,NaN,ULEZ,Hatchback,NaN,Electric,https://www.autotrader.co.uk/car-details/20200...,10665,Volkswagen 99kW e-Golf 35kWh 5dr Auto,NaN,"??22,490",Automatic,2019 (69 reg)
608,NaN,ULEZ,Hatchback,NaN,Electric,https://www.autotrader.co.uk/car-details/20201...,5086,Volkswagen Golf e-5dr Auto,NaN,"??23,000",Automatic,2019 (69 reg)
615,NaN,ULEZ,Hatchback,NaN,Electric,https://www.autotrader.co.uk/car-details/20200...,6347,Volkswagen 99kW e-Golf 35kWh 5dr Auto,NaN,"??23,290",Automatic,2020 (69 reg)
914,NaN,Non-ULEZ,Hatchback,NaN,Petrol,https://www.autotrader.co.uk/car-details/20201...,27271,Nissan Qashqai 1.2 Dig-t 115 Acenta Smrt Vsn P...,NaN,"??10,499",Manual,2017 (17 reg)


In [10]:
# update engine to 0 liter for electric cars and 1.2L for car #914
index_elec = df_clean[df_clean.engine.isna()].index
df_clean.at[index_elec, "engine"] = "0.0L"
df_clean.iloc[914,3] = "1.2L"

In [11]:
df_clean["engine"] = df_clean["engine"].apply(lambda x: float(x[:-1]))

In [12]:
df_clean["engine"].value_counts()

2.0    255
1.5    233
1.6    153
1.3    140
1.2     98
1.4     95
1.0     90
2.1     55
1.7     43
3.0     39
1.8     10
2.5     10
2.2      6
0.0      5
4.0      1
Name: engine, dtype: int64

### Remove link

In [13]:
df_clean.drop(["link"], axis=1, inplace=True)

### Parsing name into multiple extra features

#### Create "make" and "model" columns

In [14]:
df_clean["make"] = df_clean["name"].apply(lambda x:x.split()[0])
df_clean["model"] = df_clean["name"].apply(lambda x:x.split()[1])

In [15]:
df_clean["make"].value_counts()

Mercedes-Benz    248
Nissan           234
Volkswagen       215
Audi             200
Hyundai          126
BMW              109
Fiat              43
Mazda             23
Lexus             20
Mazda3             8
Mazda6             7
Name: make, dtype: int64

In [16]:
df_clean["model"].value_counts()

Qashqai        234
Golf           213
A              123
A3             120
C               90
Tucson          87
3               83
A1              79
500             43
2               42
Kona            39
A-Class         21
C-Class         14
NX              10
2L              10
CT               9
6                7
2.2L             2
99kW             2
2.0              2
Cthatchback      1
1.0              1
2.2              1
Name: model, dtype: int64

There are cases the information not correctly processed in the **make** and **model** columns, so need to clean them below

In [17]:
# Update Mazda3 and Mazda6 into correct make and model
index_mazda3 = df_clean[df_clean.make == "Mazda3"].index
df_clean.at[index_mazda3, "model"] = "Mazda3"
df_clean.at[index_mazda3, "make"] = "Mazda"

index_mazda6 = df_clean[df_clean.make == "Mazda6"].index
df_clean.at[index_mazda6, "model"] = "Mazda6"
df_clean.at[index_mazda6, "make"] = "Mazda"

index_6 = df_clean[df_clean.model=="6"].index
df_clean.at[index_6, "model"] = "Mazda6"

In [18]:
# Update model BMW 3 into 3-Series to differetiate from Mazda 3
index_3 = df_clean[df_clean.model == "3"]
index_bmw = index_3[index_3.make == "BMW"].index
index_mazda = index_3[index_3.make == "Mazda"].index
df_clean.at[index_bmw, "model"] = "3-Series"
df_clean.at[index_mazda, "model"] = "Mazda3"

In [19]:
# Update model BMW 2 into 2-Series
index_2 = df_clean[df_clean.model=="2"].index
df_clean.at[index_2, "model"] = "2-Series"

In [20]:
# Update model e-Golf
index_e_golf = df_clean[df_clean.model=="99kW"].index
df_clean.at[index_e_golf, "model"] = "e-Golf"

In [21]:
# Update model CT
index_CT = df_clean[df_clean.model == "Cthatchback"].index
df_clean.at[index_CT, "model"] = "CT"

In [22]:
# Update model A1
index_1 = df_clean[df_clean.model == "1.0"].index
df_clean.at[index_1, "model"] = "A1"

In [23]:
# Update model A and C to A-Class and C-Class
index_A = df_clean[df_clean.model == "A"].index
df_clean.at[index_A, "model"] = "A-Class"

index_C = df_clean[df_clean.model == "C"].index
df_clean.at[index_C, "model"] = "C-Class"

In [24]:
df_clean["make"].value_counts()

Mercedes-Benz    248
Nissan           234
Volkswagen       215
Audi             200
Hyundai          126
BMW              109
Fiat              43
Mazda             38
Lexus             20
Name: make, dtype: int64

In [25]:
df_clean["model"].value_counts()

Qashqai     234
Golf        213
A-Class     144
A3          120
C-Class     104
Tucson       87
A1           80
3-Series     67
500          43
2-Series     42
Kona         39
Mazda3       24
Mazda6       14
NX           10
CT           10
e-Golf        2
Name: model, dtype: int64

#### Filling missing BHP
* Find all samples which has "BHP","PS" and "kW" in the "name" column
* Extract numbers in the "name" column
* Update the column BHP with the highest number from the "BHP" or "PS" or "kW" value
* Convert PS to BHP using 1 PS = 0.98 BHP
* Convert kw to BHP for electric cars using 1 kw = 1.34 BHP

In [26]:
import re

df_clean["name_BHP"] = df_clean.name.apply(lambda x: np.max(list(map(int, re.findall(r'\d+', x)))) if "bhp" in x.lower() else np.nan)
df_clean["name_PS"] = df_clean.name.apply(lambda x: np.max(list(map(int, re.findall(r'\d+', x)))) if "ps" in x.lower() else np.nan)
df_clean["name_kW"] = df_clean.name.apply(lambda x: np.max(list(map(int, re.findall(r'\d+', x)))) if "kw" in x.lower() else np.nan)


# update from name_BHP
index_noBHP = df_clean[df_clean.BHP.isna()].index
df_clean.at[index_noBHP, "BHP"] = df_clean["name_BHP"]
index_noBHP = df_clean[df_clean.BHP.isna()].index
df_clean.at[index_noBHP, "BHP"] = df_clean["name_PS"]*0.98
index_noBHP = df_clean[df_clean.BHP.isna()].index
df_clean.at[index_noBHP, "BHP"] = df_clean["name_kW"]*1.34

df_clean.drop(["name_BHP","name_PS", "name_kW"], axis=1, inplace=True)

* Fill remaining missing values with median of the same model

In [27]:
# Highlight missing BHP to filled with median
df_clean["missing_BHP"] = df_clean["BHP"].apply(lambda x: 1 if pd.isnull(x) else 0)

In [28]:
BHP_median = {}

for model in df_clean.model.value_counts().keys():
    med = df_clean[df_clean.model == model].BHP.median()
    BHP_median[model] = med
    
BHP_median

{'Qashqai': 128.0,
 'Golf': 148.0,
 'A-Class': 120.0,
 'A3': 148.0,
 'C-Class': 191.0,
 'Tucson': 130.0,
 'A1': 114.0,
 '3-Series': 192.0,
 '500': 68.0,
 '2-Series': 190.0,
 'Kona': 118.0,
 'Mazda3': 119.0,
 'Mazda6': 143.0,
 'NX': 194.0,
 'CT': 134.0,
 'e-Golf': 132.66}

In [29]:
index_noBHP = df_clean[df_clean.BHP.isna()].index
df_clean.at[index_noBHP, "BHP"] = df_clean.model.apply(lambda x: BHP_median[x])

#### Feature engineering to create more trainable variables

In [30]:
# diffentiate from "ECO", SE", "Sport", "M Sport", "Lux", Nav/Satellite", "Bluetooth",
# "Leather", "3-door", 
df_clean["se"] = df_clean["name"].apply(lambda x: 1 if "se" in x.lower() else 0)
df_clean["se"].value_counts()

0    906
1    327
Name: se, dtype: int64

In [31]:
df_clean["s line"] = df_clean["name"].apply(lambda x: 1 if "s line" in x.lower() else 0)
df_clean["s line"].value_counts()

0    1145
1      88
Name: s line, dtype: int64

In [32]:
df_clean["sport"] = df_clean["name"].apply(lambda x: 1 if "sport" in x.lower() else 0)
df_clean["sport"].value_counts()

0    926
1    307
Name: sport, dtype: int64

In [33]:
df_clean["m_sport"] = df_clean["name"].apply(lambda x: 1 if "m sport" in x.lower() else 0)
df_clean["m_sport"].value_counts()

0    1159
1      74
Name: m_sport, dtype: int64

In [34]:
df_clean["amg"] = df_clean["name"].apply(lambda x: 1 if "amg" in x.lower() else 0)
df_clean["amg"].value_counts()

0    1072
1     161
Name: amg, dtype: int64

In [35]:
df_clean["sat_nav"] = df_clean["name"].apply(lambda x: 1 if "sat" in x.lower() or "nav" in x.lower() or "navigation" in x.lower() else 0)
df_clean["sat_nav"].value_counts()

0    1022
1     211
Name: sat_nav, dtype: int64

In [36]:
df_clean["bluetooth"] = df_clean["name"].apply(lambda x: 1 if "bluetooth" in x.lower() else 0)
df_clean["bluetooth"].value_counts()

0    1204
1      29
Name: bluetooth, dtype: int64

In [37]:
df_clean["leather"] = df_clean["name"].apply(lambda x: 1 if "leather" in x.lower() else 0)
df_clean["leather"].value_counts()

0    1210
1      23
Name: leather, dtype: int64

In [38]:
df_clean["sportback"] = df_clean["name"].apply(lambda x: 1 if "sportback" in x.lower() else 0)
df_clean["sportback"].value_counts()

0    1168
1      65
Name: sportback, dtype: int64

In [39]:
df_clean["performance"] = df_clean["name"].apply(lambda x: 1 if "performance" in x.lower() else 0)
df_clean["performance"].value_counts()

0    1219
1      14
Name: performance, dtype: int64

In [40]:
df_clean["parking"] = df_clean["name"].apply(lambda x: 1 if "camera" in x.lower()
                                            or "rear" in x.lower() 
                                            or "parking" in x.lower() 
                                            or "sensor" in x.lower() 
                                            or "dashcam" in x.lower() else 0)
df_clean["parking"].value_counts()

0    1220
1      13
Name: parking, dtype: int64

In [41]:
df_clean["gti"] = df_clean["name"].apply(lambda x: 1 if "gti" in x.lower() else 0)
df_clean["gti"].value_counts()

0    1213
1      20
Name: gti, dtype: int64

In [42]:
df_clean["gtd"] = df_clean["name"].apply(lambda x: 1 if "gtd" in x.lower() else 0)
df_clean["gtd"].value_counts()

0    1209
1      24
Name: gtd, dtype: int64

In [43]:
df_clean["gte"] = df_clean["name"].apply(lambda x: 1 if "gte" in x.lower() else 0)
df_clean["gte"].value_counts()

0    1212
1      21
Name: gte, dtype: int64

#### Filling missing owners
Fill missing values with median value

In [44]:
df_clean["owners"].fillna(df.owners.median(), inplace=True)

#### Filling missing transimission
Fill missing values with most frequent value

In [45]:
df_clean["transmission"].value_counts()

Manual       696
Automatic    536
Name: transmission, dtype: int64

In [46]:
df_clean["transmission"].fillna("Manual", inplace=True)

#### Parsing years

In [47]:
import datetime

df_clean["year_reg"] = df_clean["year"].apply(lambda x: x.split("(")[1].replace(")",""))
df_clean["year"] = df_clean["year"].apply(lambda x: int(x.split("(")[0]))

In [48]:
df_clean["age"] = datetime.date.today().year - df_clean["year"]

#### Parsing price

In [49]:
df_clean["price"] = df_clean["price"].apply(lambda x: int(x.replace("??","").replace(",","")))

#### Parsing fuel

In [50]:
df_fuel = df_clean[df_clean.fuel.isna()]
df_fuel.make

282    Fiat
Name: make, dtype: object

In [51]:
df_fuel = df_clean[df_clean.make == "Fiat"]
df_fuel.fuel.value_counts()

Petrol    42
Name: fuel, dtype: int64

In [52]:
df_clean["fuel"].fillna("Petrol", inplace=True)

### Check and save

In [53]:
df_clean.head()

,BHP,ULEZ,body,engine,fuel,mileage,name,owners,price,transmission,...,bluetooth,leather,sportback,performance,parking,gti,gtd,gte,year_reg,age
0,123.0,ULEZ,Hatchback,1.4,Petrol,34414,"Audi A1 1.4 TFSI SPORT 3d 123 BHP, SPORTS SEAT...",2.0,9980,Manual,...,0,0,0,0,1,0,0,0,17 reg,3
1,114.0,ULEZ,Hatchback,1.4,Petrol,16773,Audi A1 TFSI Sport 3dr 1.4,2.0,11500,Manual,...,0,0,0,0,0,0,0,0,67 reg,3
2,114.0,ULEZ,Hatchback,1.6,Diesel,76922,Audi A1 1.6 TDI SPORT 3d 114 BHP PARKING SENSO...,2.0,8599,Manual,...,0,0,0,0,1,0,0,0,17 reg,3
3,114.0,ULEZ,Hatchback,1.4,Petrol,10950,Audi A1 1.4 TFSI Sport 5dr,2.0,12174,Manual,...,0,0,0,0,0,0,0,0,67 reg,3
4,114.0,ULEZ,Hatchback,1.6,Diesel,58278,Audi A1 1.6 TDI Sport 3dr,2.0,9172,Manual,...,0,0,0,0,0,0,0,0,67 reg,3


In [54]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1233 entries, 0 to 1232
Data columns (total 30 columns):
BHP             1233 non-null float64
ULEZ            1233 non-null object
body            1233 non-null object
engine          1233 non-null float64
fuel            1233 non-null object
mileage         1233 non-null int64
name            1233 non-null object
owners          1233 non-null float64
price           1233 non-null int64
transmission    1233 non-null object
year            1233 non-null int64
make            1233 non-null object
model           1233 non-null object
missing_BHP     1233 non-null int64
se              1233 non-null int64
s line          1233 non-null int64
sport           1233 non-null int64
m_sport         1233 non-null int64
amg             1233 non-null int64
sat_nav         1233 non-null int64
bluetooth       1233 non-null int64
leather         1233 non-null int64
sportback       1233 non-null int64
performance     1233 non-null int64
parking         

In [55]:
df_clean.describe()

,BHP,engine,mileage,owners,price,year,missing_BHP,se,s line,sport,...,sat_nav,bluetooth,leather,sportback,performance,parking,gti,gtd,gte,age
count,1233.000000,1233.000000,1233.000000,1233.000000,1233.000000,1233.000000,1233.000000,1233.000000,1233.000000,1233.000000,...,1233.000000,1233.000000,1233.000000,1233.000000,1233.000000,1233.000000,1233.000000,1233.000000,1233.000000,1233.000000
mean,148.409311,1.616139,19593.000000,2.000811,17200.058394,2018.022709,0.313058,0.265207,0.071371,0.248986,...,0.171127,0.023520,0.018654,0.052717,0.011354,0.010543,0.016221,0.019465,0.017032,1.977291
std,51.538445,0.429369,13701.411563,0.028479,5072.432386,1.071837,0.463926,0.441622,0.257547,0.432601,...,0.376773,0.151609,0.135354,0.223559,0.105993,0.102180,0.126374,0.138208,0.129442,1.071837
min,68.000000,0.000000,10.000000,2.000000,5995.000000,2017.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,118.000000,1.300000,10665.000000,2.000000,13599.000000,2017.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
50%,134.000000,1.500000,16805.000000,2.000000,16275.000000,2018.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000
75%,158.000000,2.000000,26179.000000,2.000000,20300.000000,2019.000000,1.000000,1.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000
max,469.000000,4.000000,124395.000000,3.000000,53995.000000,2020.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,3.000000


In [56]:
df_clean.isna().sum()

BHP             0
ULEZ            0
body            0
engine          0
fuel            0
mileage         0
name            0
owners          0
price           0
transmission    0
year            0
make            0
model           0
missing_BHP     0
se              0
s line          0
sport           0
m_sport         0
amg             0
sat_nav         0
bluetooth       0
leather         0
sportback       0
performance     0
parking         0
gti             0
gtd             0
gte             0
year_reg        0
age             0
dtype: int64

In [57]:
df_clean.to_csv("autotrader_prices_cleaned.csv", index=False)